In [ ]:
import numpy as np
import re
import string
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
from nltk.stem import PorterStemmer        
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
%matplotlib inline

In [6]:
df = pd.read_excel('reviews.xlsx')
df.head()

,target,movie_link,author_id,author_name,review_subtitle,review_text
0,good,/film/178563/,/user/311869/,sochi2014,NaN,Настоящее кино. Наше кино. Прекрасная работа с...
1,good,/film/178563/,/user/1591566/,Альберт Попов,Предтеча многого,"Мало того, что сериал «Империя под ударом» (12..."
2,good,/film/178563/,/user/331395/,LennoxL,Срез жизни общества начала 20 го века,Период от реформ Александра Освободителя до пе...
3,good,/film/178563/,/user/4379604/,Ana_K,"Пусть мир погибнет, но восторжествует правосудие",Атмосферный сериал о Российской Империи начала...
4,neutral,/film/178563/,/user/178788/,Skept,Террор  Зло с непредсказуемыми последствиями,Самая большая беда современного общества это т...


In [8]:
df = df[df["target"] != "neutral"]

In [9]:
df["is_bad_review"] = df["target"].apply(lambda x: 1 if x == 'bad' else 0)
df = df[["review_text", "is_bad_review"]]
df.head()

,review_text,is_bad_review
0,Настоящее кино. Наше кино. Прекрасная работа с...,0
1,"Мало того, что сериал «Империя под ударом» (12...",0
2,Период от реформ Александра Освободителя до пе...,0
3,Атмосферный сериал о Российской Империи начала...,0
6,Это один из прекрасных фильмов вообще и гениал...,0


In [24]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    text = re.sub(r'[^0-9а-яА-ЯёЁ]+', ' ', text)
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = nltk.corpus.stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    return " ".join(text)

df_80 = (df.shape[0] * 80) // 100

In [25]:
df["review_clean"] = df["review_text"].apply(lambda x: clean_text(x))

TypeError: expected string or bytes-like object

In [ ]:
sid = SentimentIntensityAnalyzer()
df["sentiments"] = df["review_text"].apply(lambda x: sid.polarity_scores(x))
df = pd.concat([df.drop(['sentiments'], axis=1), df['sentiments'].apply(pd.Series)], axis=1)